In [1]:
import pandas as pd, numpy as np

In [47]:
file_Name = 'cases_report_Tata Power (Maithon Power Ltd)_(All States)_2024-06-19.csv'

In [48]:
def csvORexcel():
    global path
    path = "C:\\Python\\read\\"+file_Name
    try:
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            df = pd.read_excel(path)
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))

fn = file_Name

In [49]:
data0 = csvORexcel()
data0.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,Case Organization,Case District,...,Citizen Name,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,AMS ID
0,EP028YNT,14-Jun-2023,Suvidha : Affordable Sanitary Pads (Central),SH0007H9,Open,NaN,NaN,Not Suspected,JH - Tata Power (Maithon Power Ltd),Dhanbad,...,Shere Bibi,F,9.334953e+09,40.0,PG027FLX,Shere Bibi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
1,EP026CCR,24-May-2023,Aadhar Card linking to PAN Card_JH,SH0008WA,Data complete,NaN,NaN,Not Suspected,JH - Tata Power (Maithon Power Ltd),Dhanbad,...,Sanjeeb Hembram,M,8.789685e+09,34.0,PG0256GE,Hembram,amba.lal.hembram,Amba Lal Hembram,Govind Mahato,10840.0
2,EP026DE6,24-May-2023,Aadhar Card linking to PAN Card_JH,SH0008WA,Data complete,NaN,NaN,Not Suspected,JH - Tata Power (Maithon Power Ltd),Dhanbad,...,Pratima Gorai,F,9.693270e+09,34.0,PG015HRH,Pratima gorai,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
3,EP026LR3,26-May-2023,Aadhar Linking to MGNREGA Job Card,SH000A31,Data complete,NaN,NaN,Not Suspected,JH - Tata Power (Maithon Power Ltd),Dhanbad,...,Jhuma Mudi,F,9.693270e+09,35.0,PG025EB1,Jhuma mudi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
4,EP026LWK,26-May-2023,Aadhar Linking to MGNREGA Job Card,SH000A31,Data complete,NaN,NaN,Not Suspected,JH - Tata Power (Maithon Power Ltd),Dhanbad,...,Josna Modi,F,9.693270e+09,28.0,PG025EJ0,Jhuma Modi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0


In [50]:
states = {'AP':'Andhra Pradesh',
'AR':'Arunachal Pradesh',
'AS':'Assam',
'BR':'Bihar',
'BH':'Bihar',
'CT':'Chhattisgarh',
'CG':'Chhattisgarh',
'DL':'Delhi', 
'GA':'Goa',
'GJ':'Gujarat',
'HR':'Haryana',
'HP':'Himachal Pradesh',
'JH':'Jharkhand',
'KA':'Karnataka',
'KL':'Kerala',
'MP':'Madhya Pradesh',
'MH':'Maharashtra',
'MN':'Manipur',
'ML':'Meghalaya',
'MZ':'Mizoram',
'NL':'Nagaland',
'OR':'Odisha',
'PB':'Punjab',
'RJ':'Rajasthan',
'SK':'Sikkim',
'TN':'Tamil Nadu',
'TG':'Telangana',
'TR':'Tripura',
'UP':'Uttar Pradesh',
'UT':'Uttarakhand',
'WB':'West Bengal'}

# Data Cleaning

In [51]:
# Remove last row.
data0.drop(index = data0[data0.Createdon.isna()].index, inplace=True)

# Replace null values
data0['Scheme/Doc'].fillna('a', inplace=True)
data0['Citizen Name'].fillna('a', inplace=True)
data0['HD Name'].fillna('blank', inplace=True)
data0.Mobile.fillna(0, inplace=True)

# Changing status values and keeping only "Open/Submit/BR"
data0['Status'] = data0['Status'].apply(lambda x: 'Open' if x == 'Data complete' else 'Submitted' if (x=='Docket submitted' or x=='Document ready') else "Benefit Received" if x=='Scheme/Document received' else x)

# Changing Case Organization values from state initials to full state name.
data0['Case Organization'] = data0['Case Organization'].apply(lambda x: states[x[:2]])

# Renaming column "Case Organiisation" & "Case District" to "State" & "Disctrict"
data0.rename(columns={"Case Organization":"State","Case District":"District"}, inplace=True)

# Convert Mobile column from float to string for concatenation.
data0['Mobile'] = data0['Mobile'].apply(lambda x: str(x).strip())
#data0['Mobile'] = data0['Mobile'].astype('int64')
data0['Mobile'] = data0['Mobile'].astype('str')

# Checking duplicate records based on citizrn mobile number.
duplicates = data0[data0.duplicated(['Mobile'], keep='first')] # Keeping duplicate records
data0 = data0.drop(index = data0[data0.duplicated(['Mobile'], keep='first')].index) # Removing duplicate records.

# Adding column to check if HD mobile = Citizen mobile.
#data0['mobile_similarity'] = data0[['Mobile','HD Mobile']].apply(lambda x: True if x['Mobile'] == x['HD Mobile'] else False)
#data0 = data0[data0['mobile_similarity'] == False]

# Change gender from initial letter to full form.
data0['Gender'] = data0['Gender'].apply(lambda x: 'Male' if x=='M' else 'Female' if x=='F' else 'Other' if x=='O' else x)

# Convert "Createdon", "Docket Submitted Date", "Benefit received Date" column data type to Datetime format
dt_col = ['Createdon', 'Docket Submitted Date', 'Benefit received Date', 'DOB']

for col in dt_col:
    try:
        data0[col] = pd.to_datetime(data0[col], format='mixed', errors='ignore')
    except KeyError:
        continue

# Deleting records with status "Case Aborted" and "Application rejected"
rejectedDF = data0[(data0.Status == 'Case Aborted') | (data0.Status == 'Application rejected')] # Storing prev step deleted data
data0 = data0[(data0['Status'] != 'Case Aborted') & (data0['Status'] != 'Application rejected')]

data0.reset_index(inplace=True, drop=True)
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17917 entries, 0 to 17916
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Case Id                17917 non-null  object        
 1   Createdon              17917 non-null  datetime64[ns]
 2   Scheme/Doc             17917 non-null  object        
 3   Scheme/Doc GUID        17917 non-null  object        
 4   Status                 17917 non-null  object        
 5   Docket Submitted Date  16031 non-null  datetime64[ns]
 6   Benefit received Date  14217 non-null  datetime64[ns]
 7   HD Suspected Cases     17917 non-null  object        
 8   State                  17917 non-null  object        
 9   District               17917 non-null  object        
 10  Citizen GUID           17917 non-null  object        
 11  Citizen Name           17917 non-null  object        
 12  Gender                 17917 non-null  object        
 13  M

In [56]:
unique_hd_ids = data0['HD ID'].unique()
hd_id_samples = data0[data0['HD ID'].isin(unique_hd_ids)]

In [62]:
data0[~data0.index.isin(hd_id_samples.index)].sample(n =additional_sample_size)

ValueError: A negative number of rows requested. Please provide `n` >= 0.

In [63]:
# Ensure all HD IDs are included
unique_hd_ids = data0['HD ID'].unique()
hd_id_samples = data0[data0['HD ID'].isin(unique_hd_ids)]

# Calculate the required sample size (20% of the total dataset)
total_sample_size = int(0.2 * len(data0))
additional_sample_size = total_sample_size - len(hd_id_samples)

# Add additional random samples if needed
if additional_sample_size > 0:
    additional_samples = data0[~data0.index.isin(hd_id_samples.index)].sample(additional_sample_size)
    samp = pd.concat([hd_id_samples, additional_samples])
else:
    samp = hd_id_samples

# Display the final sample
samp

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,State,District,...,Citizen Name,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,AMS ID
0,EP028YNT,2023-06-14,Suvidha : Affordable Sanitary Pads (Central),SH0007H9,Open,NaT,NaT,Not Suspected,Jharkhand,Dhanbad,...,Shere Bibi,Female,9334953418.0,40.0,PG027FLX,Shere Bibi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
1,EP026CCR,2023-05-24,Aadhar Card linking to PAN Card_JH,SH0008WA,Open,NaT,NaT,Not Suspected,Jharkhand,Dhanbad,...,Sanjeeb Hembram,Male,8789685466.0,34.0,PG0256GE,Hembram,amba.lal.hembram,Amba Lal Hembram,Govind Mahato,10840.0
2,EP026DE6,2023-05-24,Aadhar Card linking to PAN Card_JH,SH0008WA,Open,NaT,NaT,Not Suspected,Jharkhand,Dhanbad,...,Pratima Gorai,Female,9693269756.0,34.0,PG015HRH,Pratima gorai,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
3,EP026LR3,2023-05-26,Aadhar Linking to MGNREGA Job Card,SH000A31,Open,NaT,NaT,Not Suspected,Jharkhand,Dhanbad,...,Jhuma Mudi,Female,9693269756.0,35.0,PG025EB1,Jhuma mudi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
4,EP026LWK,2023-05-26,Aadhar Linking to MGNREGA Job Card,SH000A31,Open,NaT,NaT,Not Suspected,Jharkhand,Dhanbad,...,Josna Modi,Female,9693269756.0,28.0,PG025EJ0,Jhuma Modi,Pratima.Gorai,Pratima Gorai,Govind Mahato,10868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17912,EP049BBE,2024-06-18,JH-khatiyan,DC0002L1,Benefit Received,2024-06-19,2024-06-19,Not Suspected,Jharkhand,DHANBAD,...,Rajkumar Mahata,Male,6200107430.0,21.0,PG02QEV2,Mahata,himal.kumar.mahato,Himal Kumar Mahato,Govind Mahato,12993.0
17913,EP049BBO,2024-06-18,JH-khatiyan,DC0002L1,Benefit Received,2024-06-19,2024-06-19,Not Suspected,Jharkhand,DHANBAD,...,Prasenjeet Mahato,Male,7766016488.0,18.0,PG03RKMU,Prasenjeet Mahato,himal.kumar.mahato,Himal Kumar Mahato,Govind Mahato,12993.0
17914,EP049BCJ,2024-06-18,JH-khatiyan,DC0002L1,Benefit Received,2024-06-19,2024-06-19,Not Suspected,Jharkhand,DHANBAD,...,Chuniya Bala,Female,7765040456.0,76.0,PG03RKNV,Chuniya Bala,himal.kumar.mahato,Himal Kumar Mahato,Govind Mahato,12993.0
17915,EP049BHN,2024-06-18,JH-khatiyan,DC0002L1,Benefit Received,2024-06-19,2024-06-19,Not Suspected,Jharkhand,DHANBAD,...,Nimai Chandra Mahato,Male,9939947334.0,47.0,PG02MSDL,Nimai chandra Mahato,himal.kumar.mahato,Himal Kumar Mahato,Govind Mahato,12993.0


In [53]:
samp.to_csv("C:\\Python\\export\\sample.csv", index = False)